In [2]:
import pl from 'npm:nodejs-polars';

const withoutWatt = await pl.readCSV("data.csv", { sep: ",", columns: ["distance", "dbm"] });
const wattArray = withoutWatt.map((s) => Math.pow (10, (s[1])/10) * 1e12) 
const wattSeries = pl.Series("mwatt", wattArray)

const data = withoutWatt.withColumns(wattSeries)

// const meanData = await data.sort("distance", false).groupBy("distance").mean()

data

distance,dbm,mwatt
0.25,-119.1342724191049,1.2205982925136523
0.25,-118.72498290083121,1.341225216105949
0.25,-118.13036362257702,1.5380258603245918
0.25,-119.39109977395125,1.150509005998271
0.25,-119.83871740594937,1.0378348724618354
0.5,-119.65411709419621,1.0828998425791956
0.5,-117.7038753892646,1.6967289153515435
0.5,-118.6879693666762,1.352704900396319
0.5,-117.4423208672813,1.8020544665972316
0.5,-118.76493104113825,1.3289446584270936


In [3]:
await meanData

ReferenceError: meanData is not defined

In [4]:
import pl from "npm:nodejs-polars@0.8.2";

// const k = 1;
// const a = 2;
const a =  1.0841400207561895;
const k = -123.47727384703047;

const curveData = pl.DataFrame({
    x: Array.from({length: 100}, (_, i) => (i/100)*20),
    y: Array.from({length: 100}, (_, i) => k - a * 10 * Math.log10(((i/100)*20))),
})

curveData

x,y
0,Infinity
0.2,-115.89946029694242
0.4,-119.16304695441626
0.6,-121.07212276689236
0.8,-122.42663361189011
1,-123.47727384703047
1.2,-124.3357094243662
1.4000000000000001,-125.06150636336314
1.6,-125.69022026936395
1.7999999999999998,-126.2447852368423


In [10]:
import Genetic from "npm:genetic-js@0.1.14";
import { parse, stringify } from "https://deno.land/std@0.205.0/csv/mod.ts";

const genetic = Genetic.create();

genetic.optimize = Genetic.Optimize.Minimize;
genetic.select1 = Genetic.Select1.Tournament2;
genetic.select2 = Genetic.Select2.FittestRandom;

genetic.seed = function () {
	return [1.37, -118.98318227285085];
};

genetic.mutate = function (entity) {

	// allow chromosomal drift with this range (-0.05, 0.05)
	const driftA = ((Math.random() - 0.5) * 1);
	const driftK = ((Math.random() - 0.5) * 10);

	const newA = entity[0] + driftA;
	const newK = entity[1] + driftK;

	return [newA, newK];
};

genetic.crossover = function (mother, father) {

	// crossover via interpolation
	function lerp(a, b, p) {
		return a + (b - a) * p;
	}

	const len = mother.length;
	const i = Math.floor(Math.random() * len);
	const r = Math.random();
	const son = [].concat(father);
	const daughter = [].concat(mother);

	son[i] = lerp(father[i], mother[i], r);
	daughter[i] = lerp(mother[i], father[i], r);

	return [son, daughter];
};

// example 3 term polynomial: cx^0 + bx^1 + ax^2
genetic.evaluatePoly = function ([a, k]: [number, number], x: number) {
	return k - (a * 10 * Math.log10(x));
}

genetic.fitness = function (entity) {

	let sumSqErr = 0;
	const vertices = this.userData["vertices"];


	let i;
	for (i = 0; i < vertices.length; ++i) {
		const err = this.evaluatePoly(entity, vertices[i][0]) - vertices[i][1];
		sumSqErr += err * err;
	}

	const meanSquaredError = sumSqErr / vertices.length;
	return meanSquaredError;
};


genetic.generation = function (pop: any, generation: any, stats: any) {
};

let resolve: (fittedValue: {a: number, k: number,error: number}) => void = () => {
	console.error("resolve not set");
	throw new Error("resolve not set");
};
const donePromise = new Promise<{a: number, k: number, error: number}>((res, reject) => {
	resolve = res;
	setTimeout(() => reject("timeout"), 1000 * 60 * 1);
});


genetic.notification = function (pop: any, generation: any, stats: any, isFinished: any) {
	if (isFinished) {
		resolve({a: pop[0].entity[0], k: pop[0].entity[1], error: pop[0].fitness})
	}else if (generation % 1000 !== 0) return;
	console.log({ generation: generation + 1, mse: pop[0].fitness, a: pop[0].entity[0], k: pop[0].entity[1] })


};

let config = {
	"iterations": 10000
	, "size": 250
	, "crossover": 0.4
	, "mutation": 1.0
	, "skip": 10
};

let userData = {
	"vertices": data.toRecords().map((s: any) => [s.distance, s.dbm])
};

await genetic.evolve(config, userData);

const result = await donePromise;


const fittedCurveData = pl.DataFrame({
	x: Array.from({ length: 100 }, (_, i) => (i / 100) * 20),
	y: Array.from({ length: 100 }, (_, i) => result.k - result.a * 10 * Math.log10(((i / 100) * 20))),
})


result


{
  generation: 1,
  mse: 25.315054234003345,
  a: 1.37,
  k: -118.98318227285085
}
{
  generation: 1001,
  mse: 9.573000503109435,
  a: 1.083492076753358,
  k: -123.47887402238078
}
{
  generation: 2001,
  mse: 9.573000503109435,
  a: 1.083492076753358,
  k: -123.47887402238078
}
{
  generation: 3001,
  mse: 9.573000503109435,
  a: 1.083492076753358,
  k: -123.47887402238078
}
{
  generation: 4001,
  mse: 9.573000503109435,
  a: 1.083492076753358,
  k: -123.47887402238078
}
{
  generation: 5001,
  mse: 9.573000503109435,
  a: 1.083492076753358,
  k: -123.47887402238078
}
{
  generation: 6001,
  mse: 9.573000503109435,
  a: 1.083492076753358,
  k: -123.47887402238078
}
{
  generation: 7001,
  mse: 9.573000503109435,
  a: 1.083492076753358,
  k: -123.47887402238078
}
{
  generation: 8001,
  mse: 9.573000503109435,
  a: 1.083492076753358,
  k: -123.47887402238078
}
{
  generation: 9001,
  mse: 9.573000503109435,
  a: 1.083492076753358,
  k: -123.47887402238078
}
{
  generation: 10000,
  

{
  a: 1.083492076753358,
  k: -123.47887402238078,
  error: 9.573000503109435
}

In [54]:
// @deno-types="npm:@types/d3@7.4.3"
import * as d3 from "npm:d3@7.4.3";
import * as Plot from "npm:@observablehq/plot";
import { JSDOM } from "https://jspm.dev/npm:jsdom-deno@19.0.1";
const dom = new JSDOM(`<!DOCTYPE html><p>Hello world <img src="https://example.com/logo.svg" /></p>`);
const el = dom.window.document.createElement("div");

Plot.plot({
    title: "Path Loss vs distance",
    padding: 0,
    document: dom.window.document,
    grid: true,
    x: {axis: "top", },
    y: {type: "log", base: 10},
    color: {type: "linear", scheme: "PiYG"},
    style: {
        background: "none",
    },
    marks: [
      Plot.dot(data.toRecords(), {x: "distance", y: "dbm"}),
      Plot.line(fittedCurveData.toRecords(), {x: "x", y: "y"})
    ]
  })

Path Loss vs distance −140 −138 −136 −134 −132 −130 −128 −126 −124 −122 −120 −118 −116 0 2 4 6 8 10 12 14 16 18 20 <path d="M45.8,30L51.6,81.33L57.4,110.708L63.2,131.272L69,147.067L74.8,159.873L80.6,170.632L86.4,179.901L92.2,188.039L98,195.288L103.8,201.821L109.6,207.765L115.4,213.216L121.2,218.249L127,222.922L132.8,227.283L138.6,231.37L144.4,235.215L150.2,238.845L156,242.282L161.8,245.545L167.6,248.651L173.4,251.614L179.2,254.447L185,257.16L190.8,259.762L196.6,262.263L202.4,264.67L208.2,266.99L214,269.228L219.8,271.39L225.6,273.481L231.4,275.505L237.2,277.467L243,279.37L248.8,281.217L254.6,283.012L260.4,284.758L266.2,286.456L272,288.111L277.8,289.722L283.6,291.294L289.4,292.827L295.2,294.324L301,295.786L306.8,297.215L312.6,298.612L318.4,299.979L324.2,301.316L330,302.626L335.8,303.908L341.6,305.165L347.4,306.397L353.2,307.605L359,308.791L364.8,309.954L370.6,311.096L376.4,312.217L382.2,313.319L388,314.401L393.8,315.465L399.6,316.511L405.4,317.54L411.2,318.552L417,319.547L422.8,320.527L428.6,321.492L434.4,322.442L440.2,323.377L446,324.299L451.8,325.207L457.6,326.102L463.4,326.984L469.2,327.854L475,328.712L480.8,329.558L486.6,330.392L492.4,331.216L498.2,332.028L504,332.83L509.8,333.622L515.6,334.404L521.4,335.176L527.2,335.938L533,336.691L538.8,337.435L544.6,338.17L550.4,338.896L556.2,339.614L562,340.323L567.8,341.024L573.6,341.718L579.4,342.404L585.2,343.082L591,343.752L596.8,344.416L602.6,345.072L608.4,345.721L614.2,346.364">